In [55]:
import numpy as np
from scipy.signal import spectrogram, lfilter
from scipy.signal.windows import hann
from scipy.fftpack import dct
import os
from matplotlib import pyplot as plt
import cv2
import librosa
import time

In [56]:
#setting eps value for converting spec to log
eps = 1e-10

In [57]:
#Helper function for compute_adi_even
def get_start_stop_indices(freq_bins, multiples=1000, num_bins=8):
    
    #find the frequency index numbers to group in bins of 1kHz width from 0-1 kHz, 1-2, till 7-8kHz
    stop_freq_idx = np.zeros((num_bins, 1))
    start_freq_idx = np.zeros((num_bins,1))

    for i in range(num_bins):
        stop_freq = multiples*(i+1)
        #to find the index to clip
        stop_freq_idx[i] = freq_bins.searchsorted(stop_freq, side='right') - 1
        if (i<num_bins-1):
            start_freq_idx[i+1] = stop_freq_idx[i]+1
        
    return start_freq_idx, stop_freq_idx

In [58]:
def compute_ndsi(spec, start_indices, stop_indices, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq):
    
    #Taken from the Kasten 2012 paper - page 6 has NDSI description
    
    #Anthrophony bin: 1-2 kHz
    start_a = int(start_indices[start_a_freq])
    stop_a = int(stop_indices[stop_a_freq-1])
    
    #Biophony bin: 2-8 kHz
    start_b = int(start_indices[start_b_freq])
    stop_b = int(stop_indices[stop_b_freq-1])
    
    #Taking absolute value - power spectral density was squared, so intensities are made positive here
    
    #anth_sum = np.sum(np.abs(spec[start_a:stop_a, :]))
    #bio_sum = np.sum(np.abs(spec[start_b:stop_b, :]))
    #NDSI = (bio_sum-anth_sum)/(bio_sum+anth_sum)
    
    anth_sum = np.sum(np.sum(np.abs(spec[:, start_a:stop_a, :]), axis = 2), axis = 1)
    bio_sum = np.sum(np.sum(np.abs(spec[:, start_b:stop_b, :]), axis = 2), axis = 1)
    NDSI = (bio_sum-anth_sum)/(bio_sum+anth_sum)
    
    return NDSI

In [59]:
#function to load spectrograms and display dimensions (optional)
def load_display_dimensions(class_name, dim=0):

    #   description of data files
    #   spec_data is like a dict with fields
    #   'specs' (contains the mel-filtered spectrograms, in linear scale, as [num_specs, num_rows, num_cols])
    #   'spec_f' (contains the frequency axis points as a 1D array)
    #   'spec_t' (contains the time axis points as a 1D array)   
    
    #choose if _spec or _melspec
    fname_load = class_name + '_spec' + '.npz'
    spec_data = np.load(fname_load)
    
    time_steps = len(spec_data['spec_t'])
    freq_bins = len(spec_data['spec_f'])
    
    dim = 0
    
    if (dim==1):
        print('File has %i specs of dimensions (%i x %i)' % (
        spec_data['specs'].shape[0], spec_data['specs'].shape[1], spec_data['specs'].shape[2]))
        print('Frequency axis points are in spec_data[\'spec_f\'] and has %i values' % freq_bins)
        print('Time axis points are in spec_data[\'spec_t\'] and has %i values' % time_steps)
        
    return spec_data, time_steps, freq_bins

In [60]:
#function to calculate all acoustic indices and plot spectrogram (optional)
def calc_plot(spec_start_idx, num_specs, spec_data, time_steps, freq_bins, freq_vals, start_freq_idx, stop_freq_idx, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot=0):

    NDSI = np.zeros((num_specs, 1))
        
    #Calculation of acoustic indices for num_spec spectrograms here
    all_specs = 10 * np.log10(spec_data['specs'][spec_start_idx:spec_start_idx+num_specs, :, :] + eps)
    #NDSI = compute_ndsi(all_specs, start_freq_idx, stop_freq_idx, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq)
      
    for spec_idx in range(num_specs):
     
        #Getting current spec from saved log spec
        current_spec = all_specs[spec_start_idx + spec_idx, :, :] 
        
        NDSI[spec_idx] = compute_ndsi(current_spec, start_freq_idx, stop_freq_idx, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq)
        
        if(plot==1):
            cax = axes[spec_idx].imshow(current_spec, cmap=plt.get_cmap('jet'), interpolation='none', origin='lower')
            fig.colorbar(cax, ax=axes[spec_idx], orientation='vertical')
        
    
    return NDSI
       

In [61]:
def calls(class_name, spec_start_idx, num_specs, multiples, num_bins, dim=0, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot=0):

    #Load the correct file
    spec_data, time_steps, freq_bins = load_display_dimensions(class_name, dim)
    
    #Calling helper function 
    start_freq_idx, stop_freq_idx = get_start_stop_indices(spec_data['spec_f'], multiples, num_bins)
    plot = 0
   
    #Computing acoustic indices
    NDSI = calc_plot(spec_start_idx, num_specs, spec_data, time_steps, freq_bins, spec_data['spec_f'], start_freq_idx, stop_freq_idx, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
        
    return NDSI

In [62]:
#environmental sounds/noise
fname_list = ["Airplane_Sound", "Heavy_rain", "Brown_Noise", "Pink_Noise", "White_Noise", "Rufous_Antpitta", "Grey_headed_woodpecker", "Italian_Sparrow", "Hawk_scream", "Dove"]



In [63]:
#Main

#start timer
tic = time.time()
  
#Make sure the indices don't get larger than spec_data['specs'].shape[0]
spec_start_idx = 0
#num_specs = 2

num_specs = 1

#Other parameters
plot = 0
dim = 1
num_bins = 8            #for compute_adi_even
multiples = 1000

#NDSI frequencies in kHz
start_a_freq = 1
stop_a_freq = 2
start_b_freq = 2
stop_b_freq = 8

NDSI = np.zeros((10, 1))
#Calls for each class

for i in range(10):
    
    #str_name = 'fname' + str(i+1)
    NDSI[i] = calls(fname_list[i], spec_start_idx, num_specs, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
    
    print("For ", fname_list[i], "the NDSI value is: ", NDSI[i])

#end timer
toc = time.time()
print("Time taken: " + str(1000*(toc-tic)) + " ms")

#Display    
#plt.show()

For  Airplane_Sound the NDSI value is:  [ 0.81170344]
For  Heavy_rain the NDSI value is:  [ 0.77215022]
For  Brown_Noise the NDSI value is:  [ 0.7483924]
For  Pink_Noise the NDSI value is:  [ 0.71737576]
For  White_Noise the NDSI value is:  [ 0.71737576]
For  Rufous_Antpitta the NDSI value is:  [ 0.75110531]
For  Grey_headed_woodpecker the NDSI value is:  [ 0.7614271]
For  Italian_Sparrow the NDSI value is:  [ 0.75381267]
For  Hawk_scream the NDSI value is:  [ 0.7130906]
For  Dove the NDSI value is:  [ 0.77158856]
Time taken: 148.9408016204834 ms
